In [1]:
# importing Mogonet files from its GitHub directory

! git clone 'https://github.com/txWang/MOGONET'

Cloning into 'MOGONET'...
remote: Enumerating objects: 124, done.
remote: Counting objects: 100% (10/10), done.
remote: Compressing objects: 100% (10/10), done.
remote: Total 124 (delta 2), reused 0 (delta 0), pack-reused 114 (from 1)
Receiving objects: 100% (124/124), 59.96 MiB | 21.70 MiB/s, done.
Resolving deltas: 100% (6/6), done.


______________________________________



**Important Note:** As some methods in the original codes are depricated, and for the purpose of our study, it is needed to replace some lines in the original code with the new suitable codes.

1. replace the code below with line 50 in *utils.py*

```
return np.ndarray.item(parameter.data.cpu().numpy())
```

2. add the following line to the end of the train_test function in *train_test.py*

```
return data_trte_list, adj_te_list, trte_idx["te"], model_dict, list(labels_trte[trte_idx["te"]])
```





___________________________________



In [2]:
# training the model and save it

%cd /content/MOGONET

from train_test import train_test

data_folder = 'ROSMAP'
view_list = [1,2,3]
num_epoch_pretrain = 500
num_epoch = 2500
lr_e_pretrain = 1e-3
lr_e = 5e-4
lr_c = 1e-3

if data_folder == 'ROSMAP':
    num_class = 2
if data_folder == 'BRCA':
    num_class = 5

data_trte_list, adj_te_list, test_idx, model_dict, test_labels = train_test(data_folder, view_list, num_class,
            lr_e_pretrain, lr_e, lr_c,
            num_epoch_pretrain, num_epoch)

/content/MOGONET


/content/MOGONET/utils.py:43: UserWarning: torch.sparse.SparseTensor(indices, values, shape, *, device=) is deprecated.  Please use torch.sparse_coo_tensor(indices, values, shape, dtype=, device=). (Triggered internally at ../torch/csrc/utils/tensor_new.cpp:641.)
  return sparse_tensortype(indices, values, x.size())



Pretrain GCNs...

Training...

Test: Epoch 0
Test ACC: 0.443
Test F1: 0.604
Test AUC: 0.245


Test: Epoch 50
Test ACC: 0.679
Test F1: 0.750
Test AUC: 0.812


Test: Epoch 100
Test ACC: 0.811
Test F1: 0.831
Test AUC: 0.889


Test: Epoch 150
Test ACC: 0.802
Test F1: 0.824
Test AUC: 0.902


Test: Epoch 200
Test ACC: 0.849
Test F1: 0.849
Test AUC: 0.903


Test: Epoch 250
Test ACC: 0.840
Test F1: 0.841
Test AUC: 0.906


Test: Epoch 300
Test ACC: 0.849
Test F1: 0.852
Test AUC: 0.907


Test: Epoch 350
Test ACC: 0.840
Test F1: 0.850
Test AUC: 0.905


Test: Epoch 400
Test ACC: 0.858
Test F1: 0.865
Test AUC: 0.919


Test: Epoch 450
Test ACC: 0.811
Test F1: 0.800
Test AUC: 0.906


Test: Epoch 500
Test ACC: 0.840
Test F1: 0.844
Test AUC: 0.919


Test: Epoch 550
Test ACC: 0.821
Test F1: 0.812
Test AUC: 0.913


Test: Epoch 600
Test ACC: 0.840
Test F1: 0.841
Test AUC: 0.912


Test: Epoch 650
Test ACC: 0.830
Test F1: 0.827
Test AUC: 0.910


Test: Epoch 700
Test ACC: 0.849
Test F1: 0.855
Test AUC: 0.91

# Sensitivity Analysis

In [3]:
import torch
import pandas as pd


def sens_calc(data_mx, weight_mx, feat_dir):
  '''
  data_mx: input data matrix with shape (n_samples, n_features)
  weight_mx: model's first layer's weight matrix with shape (n_features, n_nex_layer_nodes)
  feat_dir: directory to a file containing the feature's names

  return: a dataframe containing features' names and their importance
  '''
  tmp_data_std = torch.std(data_mx, dim=0, keepdim=True)
  tmp_weights_sum = weight_mx.sum(axis=1).abs()
  sens_values = tmp_data_std*tmp_weights_sum
  feat_names = pd.read_csv(feat_dir, header=None).values
  df = pd.DataFrame()
  df['features'] = feat_names.flatten()
  df['values'] = sens_values.detach().numpy().flatten()
  return df

In [4]:
df1 = sens_calc(data_trte_list[0], model_dict['E1'].gc1.weight, '/content/MOGONET/ROSMAP/1_featname.csv') # first modality
df2 = sens_calc(data_trte_list[1], model_dict['E2'].gc1.weight, '/content/MOGONET/ROSMAP/2_featname.csv') # second modality
df3 = sens_calc(data_trte_list[2], model_dict['E3'].gc1.weight, '/content/MOGONET/ROSMAP/3_featname.csv') # third modality

final_df = pd.concat([df1, df2, df3])
sort_df = final_df.sort_values(by=['values'], ascending=False)

sensitivity_df = sort_df.iloc[:30, :]
sensitivity_df

,features,values
86,ENSG00000266903.1,0.513394
193,ENSG00000203565.2,0.501760
134,ENSG00000228058.1,0.491581
85,ENSG00000196154.7,0.394476
44,ENSG00000188269.3,0.337663
121,ENSG00000111087.5,0.278237
186,ENSG00000260788.1,0.267225
189,ENSG00000166394.10,0.225492
24,ENSG00000178796.8,0.225397
46,ENSG00000010310.4,0.209070


In [7]:
sort_df.to_csv(r"/content/sensitivity_analysis.csv")

# Gradient Saliency

In [8]:
import numpy as np
import pandas as pd

def zero_grad_models(inp_model_dic, inp_num_view):
  '''
  a function to zero the gradients of each model in the model_dict object
  inp_model_dict: a dict object containing each component of MOGONET
  inp_num_view: the number of modalities used
  '''
  for i in range(inp_num_view):
    inp_model_dic["C{:}".format(i+1)].zero_grad()
    inp_model_dic["E{:}".format(i+1)].zero_grad()
  inp_model_dic["C"].zero_grad()


x1 = data_trte_list[0]
x1.requires_grad = True
x2 = data_trte_list[1]
x2.requires_grad = True
x3 = data_trte_list[2]
x3.requires_grad = True

c1 = model_dict["C1"](model_dict["E1"](x1,adj_te_list[0]))
c2 = model_dict["C2"](model_dict["E2"](x2,adj_te_list[1]))
c3 = model_dict["C3"](model_dict["E3"](x3,adj_te_list[2]))
output = model_dict["C"]([c1, c2, c3])


# Zero gradients for all models
num_view = 3
zero_grad_models(model_dict, num_view)

# Compute the gradients
gradients_1 = []  # save the gradients with respect to the features in the first modality
gradients_2 = []  # save the gradients with respect to the features in the second modality
gradients_3 = []  # save the gradients with respect to the features in the third modality

for i in test_idx:  # loop over the test samples
    zero_grad_models(model_dict, num_view)    # Zero the gradients for each model
    output[i, 1].backward(retain_graph=True)  # calculate score for the positive class (class=1)
    gradients_1.append(x1.grad.data[i])

    zero_grad_models(model_dict, num_view)    # Zero the gradients for each model
    output[i, 1].backward(retain_graph=True)  # calculate score for the positive class (class=1)
    gradients_2.append(x2.grad.data[i])

    zero_grad_models(model_dict, num_view)    # Zero the gradients for each model
    output[i, 1].backward(retain_graph=True)  # calculate score for the positive class (class=1)
    gradients_3.append(x3.grad.data[i])

In [9]:
def feature_df(feat_dir, gradients):
  imp_lst = np.abs(np.array(gradients)).mean(0)
  feat_names = pd.read_csv(feat_dir, header=None).values
  df = pd.DataFrame()
  df['features'] = feat_names.flatten()
  df['values'] = imp_lst
  return df

df1 = feature_df('/content/MOGONET/ROSMAP/1_featname.csv', gradients_1)
df2 = feature_df('/content/MOGONET/ROSMAP/2_featname.csv', gradients_2)
df3 = feature_df('/content/MOGONET/ROSMAP/3_featname.csv', gradients_3)

final_df = pd.concat([df1, df2, df3])
sort_df = final_df.sort_values(by=['values'], ascending=False)

gradient_saliency_df = sort_df.iloc[:30, :]
gradient_saliency_df

,features,values
64,ENSG00000168743.8,34.528091
60,hsa-miR-146b-5p,32.398628
45,ENSG00000253210.1,30.660353
54,ENSG00000142765.12,29.828281
58,cg07992625,28.894999
16,ENSG00000227392.1,28.446060
48,hsa-miR-132,26.542709
117,hsa-miR-423-3p,25.623625
135,ENSG00000180777.9,25.509007
86,cg12924262,24.096394


In [10]:
sort_df.to_csv(r"/content/gradient_saliency.csv")

# Graph Masking

In [11]:
import torch.nn.functional as F

class SoftGraphMasking:
    def __init__(self, inp_model_dict, epochs=100, lr=0.001):
        self.model_dict = inp_model_dict
        self.epochs = epochs
        self.lr = lr

    def explain(self, x_lst, adjacency_matrix, target_index):
      # x_lst: a list containing [x1, x2, x3]

        for i in self.model_dict:
          model_dict[i].eval()

        # Initialize feature mask
        torch.manual_seed(0)
        mask_1 = torch.nn.Parameter(torch.randn(x_lst[0].shape[1], requires_grad=True, device=x_lst[0].device))
        mask_2 = torch.nn.Parameter(torch.randn(x_lst[1].shape[1], requires_grad=True, device=x_lst[1].device))
        mask_3 = torch.nn.Parameter(torch.randn(x_lst[2].shape[1], requires_grad=True, device=x_lst[2].device))

        optimizer_1 = torch.optim.Adam([mask_1], lr=self.lr)
        optimizer_2 = torch.optim.Adam([mask_2], lr=self.lr)
        optimizer_3 = torch.optim.Adam([mask_3], lr=self.lr)

        for epoch in range(self.epochs):
            optimizer_1.zero_grad()
            optimizer_2.zero_grad()
            optimizer_3.zero_grad()

            # Apply mask to input features
            masked_x1 = x_lst[0] * mask_1.sigmoid()
            masked_x2 = x_lst[1] * mask_2.sigmoid()
            masked_x3 = x_lst[2] * mask_3.sigmoid()

            c1 = self.model_dict["C1"](self.model_dict["E1"](masked_x1,adjacency_matrix[0]))
            c2 = self.model_dict["C2"](self.model_dict["E2"](masked_x2,adjacency_matrix[1]))
            c3 = self.model_dict["C3"](self.model_dict["E3"](masked_x3,adjacency_matrix[2]))
            masked_output = self.model_dict["C"]([c1, c2, c3])


            c1 = self.model_dict["C1"](self.model_dict["E1"](x_lst[0],adjacency_matrix[0]))
            c2 = self.model_dict["C2"](self.model_dict["E2"](x_lst[1],adjacency_matrix[1]))
            c3 = self.model_dict["C3"](self.model_dict["E3"](x_lst[2],adjacency_matrix[2]))
            original_output = self.model_dict["C"]([c1, c2, c3])

            loss = F.binary_cross_entropy(F.sigmoid(masked_output[:,1])[target_index],
                                          F.sigmoid(original_output[:,1])[target_index])

            loss.backward()
            optimizer_1.step()
            optimizer_2.step()
            optimizer_3.step()

        print(f'Traget:{target_index}, Explainer Loss: {loss.item()}')

        feature_importance = [mask_1.sigmoid().detach().cpu().numpy(),
                              mask_2.sigmoid().detach().cpu().numpy(),
                              mask_3.sigmoid().detach().cpu().numpy()]
        return feature_importance, loss.item()

In [12]:
first_modality_feat_imp = []
second_modality_feat_imp = []
third_modality_feat_imp = []

for i in test_idx:
  feat_imp, loss = SoftGraphMasking(model_dict).explain(data_trte_list, adj_te_list, i)
  if abs(loss) <= 0.5:        # only keeping values for samples that SoftGraphMasking was able to converge
    first_modality_feat_imp.append(feat_imp[0])
    second_modality_feat_imp.append(feat_imp[1])
    third_modality_feat_imp.append(feat_imp[2])

Traget:245, Explainer Loss: 0.03492828831076622
Traget:246, Explainer Loss: 0.3682616055011749
Traget:247, Explainer Loss: 0.6897468566894531
Traget:248, Explainer Loss: 0.687370777130127
Traget:249, Explainer Loss: 0.2174566090106964
Traget:250, Explainer Loss: 0.020879428833723068
Traget:251, Explainer Loss: 0.6821285486221313
Traget:252, Explainer Loss: 0.32628923654556274
Traget:253, Explainer Loss: 0.12076659500598907
Traget:254, Explainer Loss: 0.3842940330505371
Traget:255, Explainer Loss: 0.6877633333206177
Traget:256, Explainer Loss: 0.02067263424396515
Traget:257, Explainer Loss: 0.06384316086769104
Traget:258, Explainer Loss: 0.016874738037586212
Traget:259, Explainer Loss: 0.642406165599823
Traget:260, Explainer Loss: 0.025637513026595116
Traget:261, Explainer Loss: 0.04592029005289078
Traget:262, Explainer Loss: 0.04763542860746384
Traget:263, Explainer Loss: 0.014497381635010242
Traget:264, Explainer Loss: 0.4654183089733124
Traget:265, Explainer Loss: 0.06498274207115173

In [13]:
df1 = feature_df('/content/MOGONET/ROSMAP/1_featname.csv', first_modality_feat_imp)
df2 = feature_df('/content/MOGONET/ROSMAP/2_featname.csv', second_modality_feat_imp)
df3 = feature_df('/content/MOGONET/ROSMAP/3_featname.csv', third_modality_feat_imp)

final_df = pd.concat([df1, df2, df3])
sort_df = final_df.sort_values(by=['values'], ascending=False)

gnnExplainer_df = sort_df.iloc[:30, :]
gnnExplainer_df

,features,values
20,hsa-miR-29c,0.968125
59,ENSG00000230387.1,0.967702
106,hsa-miR-370,0.967164
157,cg21663431,0.933314
69,ENSG00000182902.8,0.927916
173,hsa-miR-891b,0.927206
175,hsa-miR-1180,0.919174
124,cg17560332,0.917991
169,hsa-miR-769-5p,0.917401
141,hsa-miR-517c_hsa-miR-519a,0.911610


In [14]:
sort_df.to_csv(r"/content/graph_masking.csv")

# SHAP

In [15]:
x = torch.cat((data_trte_list[0],data_trte_list[1],data_trte_list[2]), dim=1)
means_lst = torch.mean(x, dim=0)

In [16]:
import math

def model_prediction(inp_x, inp_idx):
  '''
  inp_x: a tensor with shape (n_samples, (n_m1_feat + n_m2_feat + n_m3_feat))
  inp_idx: the target sample's index

  returns the probability of the target sample belonging to category 1 (Alzheimer)
  '''

  c1 = model_dict["C1"](model_dict["E1"](inp_x[:,:200],adj_te_list[0]))
  c2 = model_dict["C2"](model_dict["E2"](inp_x[:,200:400],adj_te_list[1]))
  c3 = model_dict["C3"](model_dict["E3"](inp_x[:,400:600],adj_te_list[2]))
  output = model_dict["C"]([c1, c2, c3])
  prob = F.softmax(output, dim=1).data.cpu().numpy()
  return prob[inp_idx][1]


# ____________________________________________________________________
def z_generator():
  # a function to generate z
  z_lst = []
  idx = []
  while(True):
    z = torch.zeros(600)
    tmp_num = torch.randint(1,11,(1,)).item() # only 1 to 10 1s
    indices = torch.randperm(600)[:tmp_num]
    if indices not in idx:  # remove repeated lists
      idx.append([indices])
      z[indices] = 1
      z_lst.append(z)
    if len(z_lst) == 50:
      break
  while(True):
    z = torch.zeros(600)
    tmp_num = torch.randint(590,600,(1,)).item()  # only 590 to 600 1s
    indices = torch.randperm(600)[:tmp_num]
    if indices not in idx:  # remove repeated lists
      idx.append([indices])
      z[indices] = 1
      z_lst.append(z)
    if len(z_lst) == 100:   # to generate 100 neighbors for each target sample
      break

  return z_lst


# ____________________________________________________________________
def neighbor_weights_predictions(inp_idx, inp_z):
  '''
  inp_idx: the target idx
  inp_z: a list containing zs
  '''
  neighbors = []      # a list containing neighbors of the target sample
  weights = []        # a list containing the weights of each neighbor
  predictions = []    # a list containing the prediction of MOGONET for the neighbors
  for zz in inp_z:
    tmp = x.clone()
    masked_x = zz * tmp[inp_idx]
    zero_idices = (masked_x == 0).nonzero(as_tuple=True)[0]
    masked_x[zero_idices] = means_lst[zero_idices]
    neighbors.append(masked_x)

    w = (599)/((math.comb(600,int(sum(zz)))) * int(sum(zz)) * (600 - int(sum(zz))))  # formula in the SHAP paper
    weights.append(w)

    tmp[inp_idx] = masked_x
    y = model_prediction(tmp, inp_idx)
    predictions.append(y)
  return torch.stack(neighbors), torch.Tensor(weights), torch.Tensor(predictions)



#_________________________________________
class weightedLR(torch.nn.Module):
  def __init__(self, inp_dim):
    super().__init__()
    self.linear = torch.nn.Linear(inp_dim, 1)
  def forward(self, x):
    return self.linear(x).squeeze()


#_______________________________________
def weighted_mse_loss(output, target, weights):
  return torch.sum(weights * (output - target) ** 2) / torch.sum(weights)


In [17]:
shap_values = []
for target_test_index in test_idx:
  z_lists = z_generator()
  nei, wei, pre = neighbor_weights_predictions(target_test_index, z_lists)
  model = weightedLR(600)
  opt = torch.optim.Adam(model.parameters(), lr=0.001)
  n_epochs = 100
  for epoch in range(n_epochs):
    model.train()
    predictions = model(nei)
    loss = weighted_mse_loss(predictions, pre, wei)
    opt.zero_grad()
    loss.backward(retain_graph=True)
    opt.step()
  print('sample number', target_test_index,'loss is:', float(loss))
  shap_values.append(np.abs(model.state_dict()['linear.weight'].detach().numpy()))


sample number 245 loss is: 0.001966757932677865
sample number 246 loss is: 0.0018178608734160662
sample number 247 loss is: 0.02465914562344551
sample number 248 loss is: 0.0239682886749506
sample number 249 loss is: 0.009052716195583344
sample number 250 loss is: 0.0037750292103737593
sample number 251 loss is: 0.020115967839956284
sample number 252 loss is: 0.040947943925857544
sample number 253 loss is: 0.06439334899187088
sample number 254 loss is: 0.0002857566869352013
sample number 255 loss is: 0.054717063903808594
sample number 256 loss is: 0.0036163185723125935
sample number 257 loss is: 0.00012781075201928616
sample number 258 loss is: 0.00012053166574332863
sample number 259 loss is: 0.09870085120201111
sample number 260 loss is: 3.60104713763576e-05
sample number 261 loss is: 0.011103815399110317
sample number 262 loss is: 0.0014645345509052277
sample number 263 loss is: 0.006440270226448774
sample number 264 loss is: 0.06850558519363403
sample number 265 loss is: 0.01768846

In [18]:
SHAP = np.mean(np.array(shap_values).squeeze(), axis=0)

df1 = pd.DataFrame()
feat_names_1 = pd.read_csv('/content/MOGONET/ROSMAP/1_featname.csv', header=None).values
df1['feature_name'] = feat_names_1.flatten()

df2 = pd.DataFrame()
feat_names_2 = pd.read_csv('/content/MOGONET/ROSMAP/2_featname.csv', header=None).values
df2['feature_name'] = feat_names_2.flatten()

df3 = pd.DataFrame()
feat_names_3 = pd.read_csv('/content/MOGONET/ROSMAP/3_featname.csv', header=None).values
df3['feature_name'] = feat_names_3.flatten()

final_df = pd.concat([df1, df2, df3])
final_df['values'] = SHAP
sort_df = final_df.sort_values(by=['values'], ascending=False)

shap_df = sort_df.iloc[:30, :]
shap_df

,feature_name,values
86,ENSG00000266903.1,0.034082
58,cg07992625,0.033528
44,ENSG00000188269.3,0.033378
45,ENSG00000253210.1,0.032640
59,ENSG00000230387.1,0.032462
83,cg12799265,0.032219
134,ENSG00000228058.1,0.031301
142,ENSG00000197704.7,0.030567
66,ENSG00000251372.1,0.030446
41,ENSG00000254239.1,0.030350


In [19]:
sort_df.to_csv(r"/content/shap.csv")

# Important Note:
As the model's weights after training and the initialization in different XAI techniques may varry in different executions, it is probable to achieve results slightly different from what we report here.